In [1]:
# License: BSD-3-Clause
# Copyright the MNE-Python contributors.

In [2]:
%pip install scipy
%pip install mne
%pip install matplotlib
%pip install numpy
%pip install pandas




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [34]:
import numpy as np
from scipy import linalg

import mne
from mne.datasets import sample
from mne.viz import plot_sparse_source_estimates

data_path = sample.data_path()
meg_path = data_path / "MEG" / "sample"
fwd_fname = meg_path / "sample_audvis-meg-eeg-oct-6-fwd.fif"
ave_fname = meg_path / "sample_audvis-ave.fif"
cov_fname = meg_path / "sample_audvis-shrunk-cov.fif"
subjects_dir = data_path / "subjects"
condition = "Left Auditory"

# Read noise covariance matrix
noise_cov = mne.read_cov(cov_fname)
# Handling average file
evoked = mne.read_evokeds(ave_fname, condition=condition, baseline=(None, 0))
evoked.crop(tmin=0.04, tmax=0.18)

evoked = evoked.pick(picks="meg", exclude="bads")
# Handling forward solution
forward = mne.read_forward_solution(fwd_fname)

    365 x 365 full covariance (kind = 1) found.
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 59) active
Reading C:\Users\Ilian\mne_data\MNE-sample-data\MEG\sample\sample_audvis-ave.fif ...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -199.80 ...     499.49 ms (Left Auditory)
        0 CTF compensation matrices available
        nave = 55 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Applying baseline correction (mode: mean)
Removing projector <Projection | Average EEG reference, active : True, n_channels : 60>
Reading forward solution from C:\Users\Ilian\mne_data\MNE-sample-data\MEG\sample\sample_audvis-meg-eeg-oct-6-fwd.fif...
    Readi

In [35]:
# Filter Gain matrix to keep only MEG channels
channel_names = forward['sol']['row_names']
indices_meg = [i for i, name in enumerate(channel_names) if name.startswith('MEG')]
indices_meg.pop()
gain_matrix = forward['sol']['data'][indices_meg, :]
print(gain_matrix.shape)


# TODO : Do not use arbitrary the first n grid points
grid_locs_matrix = forward['src'][0]['rr']
grid_locs_matrix = grid_locs_matrix[:gain_matrix_meg.shape[1], :]
print(grid_locs_matrix.shape)

# TODO : Do not use arbitrary the first n grid points
grid_ori_matrix = forward['src'][0]['nn']
grid_ori_matrix = grid_ori_matrix[:gain_matrix_meg.shape[1], :]
print(grid_ori_matrix.shape)

vertex_connectivity_matrix = mne.spatial_src_adjacency(forward['src'])
print(vertex_connectivity_matrix.shape)

datetime_matrix = evoked.times
print(datetime_matrix)
print(datetime_matrix.shape)

print(evoked.ch_names)

num_sensors = len(evoked.ch_names)
print(num_sensors)

data_matrix = evoked.data
print(data_matrix.shape)


(305, 22494)
(22494, 3)
(22494, 3)
-- number of adjacent vertices : 8196
(7498, 7498)
[0.03995904 0.041624   0.04328896 0.04495392 0.04661888 0.04828384
 0.0499488  0.05161376 0.05327872 0.05494368 0.05660864 0.0582736
 0.05993856 0.06160352 0.06326848 0.06493344 0.0665984  0.06826336
 0.06992832 0.07159328 0.07325824 0.0749232  0.07658817 0.07825313
 0.07991809 0.08158305 0.08324801 0.08491297 0.08657793 0.08824289
 0.08990785 0.09157281 0.09323777 0.09490273 0.09656769 0.09823265
 0.09989761 0.10156257 0.10322753 0.10489249 0.10655745 0.10822241
 0.10988737 0.11155233 0.11321729 0.11488225 0.11654721 0.11821217
 0.11987713 0.12154209 0.12320705 0.12487201 0.12653697 0.12820193
 0.12986689 0.13153185 0.13319681 0.13486177 0.13652673 0.13819169
 0.13985665 0.14152161 0.14318657 0.14485153 0.14651649 0.14818145
 0.14984641 0.15151137 0.15317633 0.15484129 0.15650625 0.15817121
 0.15983617 0.16150113 0.16316609 0.16483105 0.16649601 0.16816097
 0.16982593 0.17149089 0.17315585 0.17482081

C:\Users\Ilian\AppData\Local\Temp\ipykernel_716\1420139759.py:19: RuntimeWarning: 8.5% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  vertex_connectivity_matrix = mne.spatial_src_adjacency(forward['src'])


In [5]:
def apply_solver(solver, evoked, forward, noise_cov, loose=0.2, depth=0.8):
    """Call a custom solver on evoked data.

    This function does all the necessary computation:

    - to select the channels in the forward given the available ones in
      the data
    - to take into account the noise covariance and do the spatial whitening
    - to apply loose orientation constraint as MNE solvers
    - to apply a weigthing of the columns of the forward operator as in the
      weighted Minimum Norm formulation in order to limit the problem
      of depth bias.

    Parameters
    ----------
    solver : callable
        The solver takes 3 parameters: data M, gain matrix G, number of
        dipoles orientations per location (1 or 3). A solver shall return
        2 variables: X which contains the time series of the active dipoles
        and an active set which is a boolean mask to specify what dipoles are
        present in X.
    evoked : instance of mne.Evoked
        The evoked data
    forward : instance of Forward
        The forward solution.
    noise_cov : instance of Covariance
        The noise covariance.
    loose : float in [0, 1] | 'auto'
        Value that weights the source variances of the dipole components
        that are parallel (tangential) to the cortical surface. If loose
        is 0 then the solution is computed with fixed orientation.
        If loose is 1, it corresponds to free orientations.
        The default value ('auto') is set to 0.2 for surface-oriented source
        space and set to 1.0 for volumic or discrete source space.
    depth : None | float in [0, 1]
        Depth weighting coefficients. If None, no depth weighting is performed.

    Returns
    -------
    stc : instance of SourceEstimate
        The source estimates.
    """
    # Import the necessary private functions
    from mne.inverse_sparse.mxne_inverse import (
        _make_sparse_stc,
        _prepare_gain,
        _reapply_source_weighting,
        is_fixed_orient,
    )

    all_ch_names = evoked.ch_names

    # Handle depth weighting and whitening (here is no weights)
    forward, gain, gain_info, whitener, source_weighting, mask = _prepare_gain(
        forward,
        evoked.info,
        noise_cov,
        pca=False,
        depth=depth,
        loose=loose,
        weights=None,
        weights_min=None,
        rank=None,
    )

    # Select channels of interest
    sel = [all_ch_names.index(name) for name in gain_info["ch_names"]]
    M = evoked.data[sel]

    # Whiten data
    M = np.dot(whitener, M)

    n_orient = 1 if is_fixed_orient(forward) else 3
    X, active_set = solver(M, gain, n_orient)
    X = _reapply_source_weighting(X, source_weighting, active_set)

    stc = _make_sparse_stc(
        X, active_set, forward, tmin=evoked.times[0], tstep=1.0 / evoked.info["sfreq"]
    )

    return stc

In [6]:
%pip install matlabengine==24.1.2
%pip install mne

Note: you may need to restart the kernel to use updated packages.


In [42]:
# Start MATLAB engine
import matlab.engine
eng = matlab.engine.start_matlab("-desktop")

Exception ignored in: <function MatlabEngine.__del__ at 0x000001AD3F465670>
Traceback (most recent call last):
  File "c:\Users\Ilian\Documents\dev\best-python\.venv\lib\site-packages\matlab\engine\matlabengine.py", line 250, in __del__
    self.exit()
  File "c:\Users\Ilian\Documents\dev\best-python\.venv\lib\site-packages\matlab\engine\matlabengine.py", line 232, in exit
    pythonengine.closeMATLAB(self.__dict__["_matlab"])
SystemError: MATLAB process cannot be terminated.


In [43]:
# Add file to MATLAB path
eng.eval("addpath(genpath('C:/Users/Ilian/Documents/MATLAB/best-brainstorm/best'))", nargout=0)
eng.eval("addpath(genpath('C:/Users/Ilian/Documents/MATLAB/best-brainstorm/minFunc'))", nargout=0)
eng.eval("addpath(genpath('C:/Users/Ilian/Documents/dev/python-brainstorm/nirstorm-master/bst_plugin'))", nargout=0)
eng.eval("addpath(genpath('C:/Users/Ilian/Documents/dev/brainstorm3'))", nargout=0)

Load data in python

In [8]:
def print_attributes(obj, attr):
    for attribut in attr:
        if attribut in var:
            obj = obj[attribut]

    print(var.keys())

def display_keys(obj, indent = 0):
    # If the object is a list, only display the first MAX_ITEMS_LIST items
    MAX_ITEMS_LIST = 4

    """ Display keys recursively from a nested object which may contain lists or other objects.
        Indentation is used to show the level of nesting. """
    prefix = "  " * (indent * 2) + " - "
    if isinstance(obj, dict):
        for key, value in obj.items():
            print(f"{prefix}{key} : {type(value)}")
            display_keys(value, indent + 1)
    elif isinstance(obj, list):
        print(f"{prefix}List with {len(obj)} items")
        
        for index, item in enumerate(obj):
            if index < MAX_ITEMS_LIST:
                print(f"{prefix}Item {index}")
                display_keys(item, indent + 1)


# Load data in Python

In [36]:
print(gain_matrix.shape)
print(grid_locs_matrix.shape)
print(grid_ori_matrix.shape)
print(vertex_connectivity_matrix.shape)
print(evoked.times.shape)
print(data_matrix.shape)
print(noise_cov.data.shape)

(305, 22494)
(22494, 3)
(22494, 3)
(7498, 7498)
(85,)
(305, 85)
(365, 365)


In [44]:
from scipy.io import loadmat
from scipy.io import savemat
from scipy.io.matlab import mat_struct

# load matlab mat in python
MEMOptions = loadmat('./MEMOptions.breakpoint.mat', simplify_cells=True)["MEMOptions"]
HeadModel = loadmat('./HeadModel.mat', simplify_cells=True)['HeadModel']

# Update the options
HeadModel["Gain"] = {
    "matrix": gain_matrix,
    "modality": "MEG"
}
HeadModel["GridLoc"] = grid_locs_matrix
HeadModel["GridOrient"] = grid_ori_matrix
HeadModel["vertex_connectivity"] = vertex_connectivity_matrix

MEMOptions["mandatory"]["DataTime"] = evoked.times
MEMOptions["mandatory"]["Data"] = data_matrix

MEMOptions["optional"]["DataFile"] = ""
MEMOptions["optional"]["HeadModelFile"] = ""
# Baseline can be empty if NoiseCov is defined
MEMOptions["optional"]["Baseline"] = []
MEMOptions["optional"]["BaselineTime"] = []
MEMOptions["optional"]["TimeSegment"] = [evoked.times[5], evoked.times[6]]
MEMOptions["optional"]["Channel"] = []
MEMOptions["optional"]["ChannelFlag"] = []

MEMOptions["automatic"] = []

MEMOptions["solver"]["NoiseCov"] = noise_cov.data
MEMOptions["solver"]["NoiseCov_recompute"] = 0


# Default parameters
MEMOptions["optional"]["active_mean_method"] = 2
MEMOptions["optional"]["alpha_method"] = 3
MEMOptions["optional"]["alpha_threshold"] = 0
MEMOptions["optional"]["initial_lambda"] = 1
MEMOptions["optional"]["depth_weigth_MNE"] = 0
MEMOptions["optional"]["depth_weigth_MEM"] = 0


# Save the variables in new .mat files
savemat('./Modified_MEMOptions.mat', {'MEMOptions': MEMOptions})
savemat('./Modified_HeadModel.mat', {'HeadModel': HeadModel})

# Load the variables in the MATLAB workspace
eng.eval("load('Modified_MEMOptions.mat')", nargout=0)
eng.eval("load('Modified_HeadModel.mat')", nargout=0)

# Fix variables (Load / save mat changes the structure. Need to fix it after loading it in MATLAB)
l = "{" + "'MEG' " * num_sensors +  "}"
eng.eval(f"MEMOptions.mandatory.ChannelTypes = " + l, nargout=0)
eng.eval("MEMOptions.mandatory.DataTypes = {'MEG'};", nargout=0)
eng.eval("MEMOptions.clustering.neighborhood_order = 4;", nargout=0)
eng.eval("MEMOptions.solver.inactive_var_mult = 0;", nargout=0)

# Load data in matlab 

In [165]:
# load Data
eng.eval("MEMOptions = importdata('./MEMOptions.breakpoint.mat')", nargout=0)
eng.eval("load('./OPTIONS.mat')", nargout=0)
eng.eval("load('./HeadModel.mat')", nargout=0)
eng.eval("VertConn = importdata('./VertConn.mat').VertConn", nargout=0)


# Change the gain matrix to a struct
eng.eval("""
    temp_struct = struct('matrix', HeadModel.Gain, 'modality', 'MEG');
    HeadModel.Gain = repmat(temp_struct, 1, 1);
""", nargout=0)

# Add the vertex connectivity to the head model
eng.eval("""
    HeadModel.vertex_connectivity = VertConn
""", nargout=0)

# Only use the first n time points (for testing, to speed up computation)
eng.eval("""
    MEMOptions.mandatory.DataTime = MEMOptions.mandatory.DataTime(1:20);
""", nargout=0)

In [10]:
def solver(M, G, n_orient):
    """Run L2 penalized regression and keep 10 strongest locations.

    Parameters
    ----------
    M : array, shape (n_channels, n_times)
        The whitened data.
    G : array, shape (n_channels, n_dipoles)
        The gain matrix a.k.a. the forward operator. The number of locations
        is n_dipoles / n_orient. n_orient will be 1 for a fixed orientation
        constraint or 3 when using a free orientation model.
    n_orient : int
        Can be 1 or 3 depending if one works with fixed or free orientations.
        If n_orient is 3, then ``G[:, 2::3]`` corresponds to the dipoles that
        are normal to the cortex.

    Returns
    -------
    X : array, (n_active_dipoles, n_times)
        The time series of the dipoles in the active set.
    active_set : array (n_dipoles)
        Array of bool. Entry j is True if dipole j is in the active set.
        We have ``X_full[active_set] == X`` where X_full is the full X matrix
        such that ``M = G X_full``.
    """
    # # Convert numpy arrays to MATLAB arrays
    # M_mat = matlab.double(M.tolist())
    # G_mat = matlab.double(G.tolist())

    # # Transfer M and G to MATLAB workspace
    # eng.workspace['M'] = M_mat
    # eng.workspace['G'] = G_mat
    # eng.workspace['n_orient'] = n_orient

    # Prepare and call the MATLAB function using the variables in the workspace
    eng.eval("be_main_call_return_struct(HeadModel, MEMOptions)", nargout=0)

    variables = eng.eval('who', nargout=1)
    print(variables) # ['HeadModel', 'MEMOptions', 'OPTIONS', 'Results', 'VertConn', 'ans', 'temp_struct']

    ImageGridAmp = eng.eval("Results.ImageGridAmp", nargout=1)

    import numpy as np
    ImageGridAmp_np = np.array(ImageGridAmp)

    # Extract the results
    X = ImageGridAmp_np

    # TODO : Extract the active set, do not create a np.ones array
    return X, np.ones(22494, dtype=bool)
print("ok")

ok


In [45]:
# loose, depth = 0.2, 0.8  # corresponds to loose orientation
loose, depth = 1.0, 0.0  # corresponds to free orientation
stc = apply_solver(solver, evoked, forward, noise_cov, loose, depth)

info["bads"] and noise_cov["bads"] do not match, excluding bad channels from both
Computing inverse operator with 305 channels.
    305 out of 366 channels remain after picking
Selected 305 channels
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 3)
Computing rank from covariance with rank=None
    Using tolerance 3.5e-13 (2.2e-16 eps * 305 dim * 5.2  max singular value)
    Estimated rank (mag + grad): 302
    MEG: rank 302 computed from 305 data channels with 3 projectors
    Setting small MEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.


MatlabExecutionError: 
  File C:\Users\Ilian\Documents\MATLAB\best-brainstorm\best\msp\be_msp.m, line 81, in be_msp

  File C:\Users\Ilian\Documents\MATLAB\best-brainstorm\best\cmem\clusters\be_stable_clustering_multim.m, line 117, in be_stable_clustering_multim

  File C:\Users\Ilian\Documents\MATLAB\best-brainstorm\best\cmem\clusters\be_cmem_clusterize_multim.m, line 80, in be_cmem_clusterize_multim

  File C:\Users\Ilian\Documents\MATLAB\best-brainstorm\best\main\be_main_clustering.m, line 56, in be_main_clustering

  File C:\Users\Ilian\Documents\MATLAB\best-brainstorm\best\cmem\solver\be_cmem_solver.m, line 161, in be_cmem_solver

  File C:\Users\Ilian\Documents\MATLAB\best-brainstorm\best\main\be_main_call.m, line 157, in be_main_call

  File C:\Users\Ilian\Documents\MATLAB\best-brainstorm\best\main\be_main_call_return_struct.m, line 2, in be_main_call_return_struct
Index exceeds the number of array elements. Index must not exceed 3.


In [ ]:
# plot_sparse_source_estimates(forward["src"], stc, bgcolor=(1, 1, 1), opacity=0.1)